In [1]:
%%time

import Halotools as ht # this is the Halotools for Abacus which imports catalogues in a unified format, not astropy.halotools
import numpy as np
import os
from halotools.mock_observables import rp_pi_tpcf_jackknife
from halotools.sim_manager import FakeSim
from abacus_baofit import baofit_data
# catalogue parameters
sim_name_prefix = 'emulator_1100box_planck'
cosmologies = [0] # 'all'
redshifts = [0.700] #'all'
products_dir = r'/mnt/gosling2/bigsim_products/emulator_1100box_planck_products/'
phases =  [0] # list(range(16)) # 'all'
halo_type = 'Rockstar'
cats = ht.make_catalogs(sim_name = sim_name_prefix, cosmologies = cosmologies, redshifts = redshifts, products_dir = products_dir, phases = phases, halo_type = halo_type,
                        load_halo_ptcl_catalog = False, # this loads subsamples, does not work
                        load_ptcl_catalog = False, # this loads uniform subsamples, not implemented
                        load_pids = 'auto')

Importing catalogue: /mnt/gosling2/bigsim_products/emulator_1100box_planck_products/emulator_1100box_planck_00-0_products/emulator_1100box_planck_00-0_rockstar_halos/z0.700
CPU times: user 8.25 s, sys: 5.38 s, total: 13.6 s
Wall time: 15 s


In [2]:
cat = cats[0][0][0]
cat.redshift
cat.halo_table['halo_mvir'] = cat.halo_table['halo_m']
cat.halo_table['halo_rvir'] = cat.halo_table['halo_r']
from halotools.utils import add_halo_hostid
add_halo_hostid(cat.halo_table, delete_possibly_existing_column = True)
cat.halo_table['halo_nfw_conc'] = cat.halo_table['halo_rvir'] / cat.halo_table['halo_klypin_rs'] # klypin_rs is more stable
# cat.halo_table.keys()

In [39]:
from halotools.empirical_models import HodModelFactory, TrivialPhaseSpace, NFWPhaseSpace, Zheng07Cens, Zheng07Sats
cens_occ_model = Zheng07Cens(redshift = cat.redshift)
cens_prof_model = TrivialPhaseSpace(redshift = cat.redshift)
sats_occ_model = Zheng07Sats(redshift = cat.redshift)
sats_prof_model = NFWPhaseSpace(redshift = cat.redshift, conc_mass_model= 'dutton_maccio14')
model = HodModelFactory(centrals_occupation = cens_occ_model, 
                        centrals_profile = cens_prof_model, 
                        satellites_occupation = sats_occ_model, 
                        satellites_profile = sats_prof_model)
model.populate_mock(cat, Num_ptcl_requirement = 10000)

In [3]:
from halotools.empirical_models import PrebuiltHodModelFactory
models = {}
models['zheng07'] = PrebuiltHodModelFactory('zheng07', redshift = cat.redshift)
models['zheng07'].populate_mock(cat, Num_ptcl_requirement = 10)
# model_instance.param_dict['logMmin'] = 12.1
# model_instance.mock.populate()

In [5]:
models['zheng07'].mock.galaxy_table.keys()

['halo_upid',
 'halo_hostid',
 'conc_NFWmodel',
 'halo_y',
 'halo_x',
 'halo_z',
 'halo_vx',
 'halo_vy',
 'halo_vz',
 'halo_rvir',
 'halo_mvir',
 'halo_id',
 'halo_num_centrals',
 'halo_num_satellites',
 'gal_type',
 'vx',
 'host_centric_distance',
 'vy',
 'y',
 'x',
 'vz',
 'z']